<a href="https://colab.research.google.com/github/nadiajelani/football_analytics/blob/main/deform_final'25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install OpenCV if not already installed
!pip install opencv-python-headless


In [ ]:
# Import libraries
import cv2
import numpy as np
import os
from google.colab import drive  # Import drive to mount Google Drive
# Mount Google Drive
drive.mount('/content/drive')


In [ ]:
image_folder = '/content/drive/MyDrive/Data and Videos/Ball 1/Drop 1'


In [ ]:
# List to store image file names (assuming sequential names like frame1.bmp, frame2.bmp, etc.)
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.bmp')])
if not image_files:
    raise ValueError(f"No .bmp files found in {image_folder}. Please check the folder path and ensure .bmp files exist.")

# Read the first image to initialize
frame = cv2.imread(image_files[0], cv2.IMREAD_GRAYSCALE)
if frame is None:
    raise ValueError(f"Failed to load the first image: {image_files[0]}")


In [ ]:
# Preprocess the first frame
_, thresh = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY_INV)

# Find contours to get initial object position
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(largest_contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = frame.shape[1] // 2, frame.shape[0] // 2  # Default center if no contour
else:
    cx, cy = frame.shape[1] // 2, frame.shape[0] // 2

# Initialize Kalman Filter (4 states: x, y, dx, dy; 2 measurements: x, y)
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
kalman.statePre = np.array([[cx], [cy], [0], [0]], np.float32)  # Initial state


In [ ]:
# Apply Canny edge detection to find the surface line
edges = cv2.Canny(frame, 50, 150, apertureSize=3)

# Use Hough Transform to detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

# Find the horizontal line closest to the bottom (assuming it's the surface)
surface_y = frame.shape[0]  # Default to bottom if no line is found
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(y1 - y2) < 10:  # Horizontal line (small y difference)
            surface_y = min(y1, y2)  # Take the lowest y as the surface
            break

print(f"Detected surface at y = {surface_y}")

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# Define image folder and load image files
image_folder = '/content/drive/MyDrive/Data and Videos/Ball 1/Drop 1'
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.bmp')])
if not image_files:
    raise ValueError(f"No .bmp files found in {image_folder}. Please check the folder path and ensure .bmp files exist.")

# Read the first image to initialize
frame = cv2.imread(image_files[0], cv2.IMREAD_GRAYSCALE)
if frame is None:
    raise ValueError(f"Failed to load the first image: {image_files[0]}")

# Detect surface line using Hough Transform
edges = cv2.Canny(frame, 50, 150, apertureSize=3)
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
surface_y = frame.shape[0]  # Default to bottom if no line is found
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(y1 - y2) < 10:  # Horizontal line (small y difference)
            surface_y = min(y1, y2)  # Take the lowest detected horizontal line

print(f"Detected surface at y = {surface_y}")

# Kalman Filter Setup for Stable Circle Detection
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.02  # Increased smoothness
# Initialize Kalman Filter state with default values (will be updated with first detection)
kalman.statePre = np.array([[frame.shape[1] // 2], [frame.shape[0] // 2], [0], [0]], np.float32)

# Initialize Variables
D_original = None
last_valid_radius = None
last_valid_contact_points = None  # To track previous frame's contact points
data_records = []
dot_records = []  # To store dot positions

# Process frames 1450 to 1650
start_frame = 1450
end_frame = 1823
for i, img_file in enumerate(image_files):
    if i < start_frame or i >= end_frame:
        continue

    # Read and preprocess current frame
    frame = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    if frame is None:
        print(f"Failed to load {img_file}")
        continue

    # Apply Gaussian blur and thresholding
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Detect circle using Hough Transform
    circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=50, param2=30, minRadius=20, maxRadius=100)

    # Predict using Kalman Filter
    prediction = kalman.predict()
    predicted_cx, predicted_cy = int(prediction[0]), int(prediction[1])

    cx, cy, radius = None, None, None
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            cx, cy, radius = circle  # Extract circle center and radius
            break  # Only take the first detected circle

        # Stabilize circle detection using Kalman Filter
        if last_valid_radius is not None:
            radius = int(0.85 * last_valid_radius + 0.15 * radius)  # Smooth transition
        last_valid_radius = radius  # Update last valid radius
        kalman.correct(np.array([[np.float32(cx)], [np.float32(cy)]]))
    else:
        # Use predicted values if no circle is detected
        cx, cy = predicted_cx, predicted_cy
        radius = last_valid_radius if last_valid_radius is not None else 50  # Fallback radius

    # Convert frame to color for drawing
    frame_color = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

    # Initialize dot positions as None
    top_dot_x, top_dot_y = None, None
    bottom_dot_x, bottom_dot_y = None, None
    contact_dot1_x, contact_dot1_y = None, None
    contact_dot2_x, contact_dot2_y = None, None
    diameter = 0
    contact_length = 0

    if radius is not None:
        # Define Top and Bottom Points
        top_point = (cx, cy - int(radius))
        bottom_point = (cx, cy + int(radius))

        # Ensure bottom point does not go below surface_y
        if bottom_point[1] >= surface_y:
            bottom_point = (cx, surface_y)

        # Store Top and Bottom Dot positions
        top_dot_x, top_dot_y = top_point
        bottom_dot_x, bottom_dot_y = bottom_point

        # Compute Diameter
        diameter = bottom_point[1] - top_point[1]
        diameter = max(diameter, 20)  # Avoid unrealistic values
        print(f"Frame {i + 1}: Diameter = {diameter} pixels")

        # Store initial diameter before deformation
        if D_original is None and bottom_point[1] < surface_y - 50:
            D_original = diameter
            print(f"Frame {i + 1}: D_original set to {D_original} pixels")

        # Draw Stable Blue Circle
        cv2.circle(frame_color, (cx, cy), int(radius), (255, 0, 0), 1)

        # Detect Contact Points (Yellow Dots)
        contact_point1 = None
        contact_point2 = None
        if bottom_point[1] == surface_y:  # When ball is in contact
            delta_y = surface_y - cy
            discriminant = radius**2 - delta_y**2
            if discriminant >= 0:
                sqrt_disc = np.sqrt(discriminant)
                x1 = int(cx - sqrt_disc)
                x2 = int(cx + sqrt_disc)
                contact_point1 = (x1, surface_y)
                contact_point2 = (x2, surface_y)
                contact_length = x2 - x1
                print(f"Contact detected! Contact Length: {contact_length} pixels")

                # Validate new contact points
                if last_valid_contact_points is not None:
                    x1_prev, x2_prev = last_valid_contact_points
                    if abs(x1 - x1_prev) > 10 or abs(x2 - x2_prev) > 10:
                        x1, x2 = x1_prev, x2_prev  # Use previous frame's values if new ones are unstable

                last_valid_contact_points = (x1, x2)  # Update valid points

                # Store Contact Dot positions
                contact_dot1_x, contact_dot1_y = contact_point1
                contact_dot2_x, contact_dot2_y = contact_point2

                # Draw Contact Dots in Yellow
                cv2.circle(frame_color, contact_point1, 3, (0, 255, 255), -1)
                cv2.circle(frame_color, contact_point2, 3, (0, 255, 255), -1)

    # Draw Surface Line (Green)
    cv2.line(frame_color, (0, surface_y), (frame.shape[1], surface_y), (0, 255, 0), 1)

    # Draw Top & Bottom Dots for Diameter (Cyan)
    if top_point is not None and bottom_point is not None:
        cv2.circle(frame_color, top_point, 3, (255, 255, 0), -1)
        cv2.circle(frame_color, bottom_point, 3, (255, 255, 0), -1)

    # Save Data for Original CSV (Diameter and Contact Length)
    data_records.append([i + 1, cx, cy, diameter, contact_length])

    # Save Dot Positions for New CSV
    dot_records.append([i + 1, top_dot_x, top_dot_y, bottom_dot_x, bottom_dot_y,
                        contact_dot1_x, contact_dot1_y, contact_dot2_x, contact_dot2_y])

    # Display Frame
    cv2_imshow(frame_color)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
2
cv2.destroyAllWindows()



CSV

In [ ]:
# Save Data to CSV
df = pd.DataFrame(data_records, columns=["Frame", "Ball_X", "Ball_Y", "Diameter", "Contact_Length"])
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drop1.csv", index=False)

# Create New CSV for Dot Positions
dot_df = pd.DataFrame(dot_records, columns=["Frame",
                                            "Top_Dot_X", "Top_Dot_Y",
                                            "Bottom_Dot_X", "Bottom_Dot_Y",
                                            "Contact_Dot1_X", "Contact_Dot1_Y",
                                            "Contact_Dot2_X", "Contact_Dot2_Y"])
dot_df.to_csv("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drop1_positions.csv", index=False)


# Display the Dot Positions Table (First 10 Frames)
print("\nTable of Dot Positions (First 10 Frames):")
print(dot_df.head(10))


MIN VS MAX

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop1.csv")  # Replace with your actual filename


# Calculate Contact Length stats
max_contact = df["Contact_Length"].max()
min_contact = df["Contact_Length"].min()

# Calculate Diameter stats
max_diameter = df["Diameter"].max()
min_diameter = df["Diameter"].min()

# Display the results
print(f"Max Contact Length: {max_contact} pixels")
print(f"Min Contact Length: {min_contact} pixels")
print(f"Max Diameter: {max_diameter} pixels")
print(f"Min Diameter: {min_diameter} pixels")

GRAPHS

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

import glob

import os

# Path to CSV files

csv_files = glob.glob("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/*.csv")  # Update path

# Plot Contact Length and Diameter

plt.figure(figsize=(12, 6))

for file in csv_files:

    df = pd.read_csv(file)

    label = os.path.basename(file).replace(".csv", "")

    # Filter out contact length <= 0 if needed

    df = df[df["Contact_Length"] > 0]

    # Plot contact length

    plt.plot(df["Frame"], df["Contact_Length"], label=f"{label} - Contact_lenght", linestyle='--')

    # Plot diameter (optional, on same graph)

 #   plt.plot(df["Frame"], df["Diameter"], label=f"{label} - Diameter")

plt.xlabel("Frame Number")

plt.ylabel("Pixels")

plt.title("Contact Length Over Time (Multiple CSVs)")

plt.legend()

plt.grid(True)

plt.tight_layout()

plt.savefig("multi_csv_comparison.png")

plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

# Load CSVs
csv_files = glob.glob("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/*.csv")

plt.figure(figsize=(14, 6))

for file in csv_files:
    df = pd.read_csv(file)
    label = os.path.basename(file).replace(".csv", "")

    # Filter out invalid data
    df = df[df["Contact_Length"] > 0]

    # Normalize frame to start from 0 for each run
    df["Relative_Frame"] = df["Frame"] - df["Frame"].iloc[0]

    # Plot Contact Length (dashed line)
    plt.plot(df["Relative_Frame"], df["Contact_Length"], label=f"{label} - Contact_Length", linestyle='--')

    # Plot Diameter (dotted line, optional)
    plt.plot(df["Relative_Frame"], df["Diameter"], label=f"{label} - Diameter", linestyle=':')

plt.xlabel("Frame Number (Relative Start = 0)")
plt.ylabel("Pixels")
plt.title("Overlapping Contact Length and Diameter (Aligned Start)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("overlapping_aligned_graphs.png")
plt.show()

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

import glob

import os

# Settings

window_size = 5  # You can increase this for more smoothing

# Load all CSVs

csv_files = glob.glob("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/*.csv")

# Contact Length Plot (Smoothed)

plt.figure(figsize=(14, 6))

for file in csv_files:

    df = pd.read_csv(file)

    label = os.path.basename(file).replace(".csv", "")

    df = df[df["Contact_Length"] > 0]

    df["Relative_Frame"] = df["Frame"] - df["Frame"].iloc[0]

    # Smooth the Contact_Length

    df["Smoothed_Contact"] = df["Contact_Length"].rolling(window=window_size, min_periods=1).mean()

    plt.plot(df["Relative_Frame"], df["Smoothed_Contact"], linestyle='--', label=f"{label} - Contact_Length")

plt.title(f"Smoothed Contact Length Over Time (Window = {window_size})")

plt.xlabel("Relative Frame Number (starting at 0)")

plt.ylabel("Contact Length (pixels)")

plt.legend()

plt.grid(True)

plt.tight_layout()

plt.savefig("smoothed_contact_length.png")

plt.show()


# Diameter Plot (Smoothed)

plt.figure(figsize=(14, 6))

for file in csv_files:

    df = pd.read_csv(file)

    label = os.path.basename(file).replace(".csv", "")

    df = df[df["Contact_Length"] > 0]

    df["Relative_Frame"] = df["Frame"] - df["Frame"].iloc[0]

    # Smooth the Diameter

    df["Smoothed_Diameter"] = df["Diameter"].rolling(window=window_size, min_periods=1).mean()

    plt.plot(df["Relative_Frame"], df["Smoothed_Diameter"], linestyle=':', label=f"{label} - Diameter")

plt.title(f"Smoothed Diameter Over Time (Window = {window_size})")

plt.xlabel("Relative Frame Number (starting at 0)")

plt.ylabel("Diameter (pixels)")

plt.legend()

plt.grid(True)

plt.tight_layout()

plt.savefig("smoothed_diameter.png")

plt.show()



DIAMETER

SMOOTH

INBOUND VELOCITY , OUTBOUND VELOCITY AND COR DROP 1

In [ ]:
import pandas as pd

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def calculate_metrics(file_path, fps, mm_per_pixel):

    # Load data from CSV file

    try:

        df = pd.read_csv(file_path)

    except FileNotFoundError:

        print(f"Error: File '{file_path}' not found. Please ensure the CSV file exists.")

        return None

    except Exception as e:

        print(f"Error loading file: {str(e)}")

        return None

    # Identify contact frames based on Contact_Length

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    if contact_frames.empty:

        print("No contact frames detected.")

        return None

    contact_start = contact_frames.min()  # Frame 1554

    contact_end = contact_frames.max()    # Frame 1625

    print(f"Contact detected from frame {contact_start} to {contact_end}")

    # Inbound Velocity (before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    if len(pre_contact_df) < 100:

        print("Not enough pre-contact frames for a 100-frame window.")

        return None

    # Use a 100-frame window before contact

    frame1 = pre_contact_df['Frame'].iloc[-100]  # 100 frames before contact

    frame2 = pre_contact_df['Frame'].iloc[-1]    # Last frame before contact

    y1 = pre_contact_df[pre_contact_df['Frame'] == frame1]['Ball_Y'].iloc[0]

    y2 = pre_contact_df[pre_contact_df['Frame'] == frame2]['Ball_Y'].iloc[0]

    t1 = frame1 / fps

    t2 = frame2 / fps

    delta_t = t2 - t1

    delta_y = (y2 - y1) * mm_per_pixel / 1000  # Convert to meters (positive downward)

    inbound_velocity = delta_y / delta_t  # m/s, positive for downward

    if inbound_velocity <= 0:

        print("Warning: Inbound velocity is not positive, possibly insufficient motion detected.")

        # Fallback: try from start of data

        frame1 = pre_contact_df['Frame'].iloc[0]

        delta_t = (frame2 - frame1) / fps

        y1 = pre_contact_df[pre_contact_df['Frame'] == frame1]['Ball_Y'].iloc[0]

        delta_y = (y2 - y1) * mm_per_pixel / 1000

        inbound_velocity = delta_y / delta_t

        print(f"Fallback Inbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={inbound_velocity:.3f} m/s")

    else:

        print(f"Inbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={inbound_velocity:.3f} m/s")

    # Outbound Velocity (after contact)

    post_contact_df = df[df['Frame'] > contact_end]

    if len(post_contact_df) < 100:

        print("Not enough post-contact frames for a 100-frame window.")

        return None

    # Use a 100-frame window after contact

    frame1 = post_contact_df['Frame'].iloc[0]    # First frame after contact

    frame2 = post_contact_df['Frame'].iloc[99]   # 100 frames after

    y1 = post_contact_df[post_contact_df['Frame'] == frame1]['Ball_Y'].iloc[0]

    y2 = post_contact_df[post_contact_df['Frame'] == frame2]['Ball_Y'].iloc[0]

    t1 = frame1 / fps

    t2 = frame2 / fps

    delta_t = t2 - t1

    delta_y = (y2 - y1) * mm_per_pixel / 1000  # Convert to meters (negative upward)

    outbound_velocity = delta_y / delta_t  # m/s, negative for upward

    if delta_y >= 0:

        print("Warning: Outbound velocity should be negative (upward motion, y decreasing).")

        # Fallback: use full post-contact range

        frame2 = post_contact_df['Frame'].iloc[-1]

        y2 = post_contact_df[post_contact_df['Frame'] == frame2]['Ball_Y'].iloc[0]

        delta_t = (frame2 - frame1) / fps

        delta_y = (y2 - y1) * mm_per_pixel / 1000

        outbound_velocity = delta_y / delta_t

        print(f"Fallback Outbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={outbound_velocity:.3f} m/s")

    else:

        print(f"Outbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={outbound_velocity:.3f} m/s")

    # Coefficient of Restitution (COR)

    # COR = -v_out / v_in (since v_out is negative and v_in is positive)

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    if cor > 1:

        print(f"Warning: COR ({cor:.3f}) > 1 is physically implausible. Adjusting to maximum realistic value of 1.")

        cor = 1.0

    elif cor < 0:

        print(f"Warning: COR ({cor:.3f}) < 0 is invalid. Setting to 0.")

        cor = 0.0

    print(f"COR = {cor:.3f}")

    return {

        "inbound_velocity": inbound_velocity,

        "outbound_velocity": outbound_velocity,

        "cor": cor

    }

# Specify the file path

file_path = "/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop1.csv"

# Run calculation

result = calculate_metrics(file_path, FPS, MM_PER_PIXEL)

# Print results

if result:

    print("\nResults:")

    print(f"Inbound Velocity: {result['inbound_velocity']:.3f} m/s (downward)")

    print(f"Outbound Velocity: {result['outbound_velocity']:.3f} m/s (upward)")

    print(f"Coefficient of Restitution (COR): {result['cor']:.3f}")


GRAPH

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def plot_ball_motion(file_path, fps, mm_per_pixel):

    # Load data from CSV file

    df = pd.read_csv(file_path)

    # Identify contact frames

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    contact_start = contact_frames.min()

    contact_end = contact_frames.max()

    # Inbound Velocity (100 frames before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    frame1_in = pre_contact_df['Frame'].iloc[-100]

    frame2_in = pre_contact_df['Frame'].iloc[-1]

    y1_in = pre_contact_df[pre_contact_df['Frame'] == frame1_in]['Ball_Y'].iloc[0]

    y2_in = pre_contact_df[pre_contact_df['Frame'] == frame2_in]['Ball_Y'].iloc[0]

    delta_t_in = (frame2_in - frame1_in) / fps

    delta_y_in = (y2_in - y1_in) * mm_per_pixel / 1000

    inbound_velocity = delta_y_in / delta_t_in

    # Outbound Velocity (100 frames after contact)

    post_contact_df = df[df['Frame'] > contact_end]

    frame1_out = post_contact_df['Frame'].iloc[0]

    frame2_out = post_contact_df['Frame'].iloc[99]

    y1_out = post_contact_df[post_contact_df['Frame'] == frame1_out]['Ball_Y'].iloc[0]

    y2_out = post_contact_df[post_contact_df['Frame'] == frame2_out]['Ball_Y'].iloc[0]

    delta_t_out = (frame2_out - frame1_out) / fps

    delta_y_out = (y2_out - y1_out) * mm_per_pixel / 1000

    outbound_velocity = delta_y_out / delta_t_out

    # COR

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    # Plot

    plt.figure(figsize=(12, 6))

    plt.plot(df['Frame'], df['Ball_Y'], 'b-', label='Ball Position (Ball_Y)')

    plt.axvspan(contact_start, contact_end, color='gray', alpha=0.3, label='Contact Period')

    plt.axvline(frame1_in, color='green', linestyle='--', label='Inbound Start')

    plt.axvline(frame2_in, color='green', linestyle='--')

    plt.axvline(frame1_out, color='red', linestyle='--', label='Outbound Start')

    plt.axvline(frame2_out, color='red', linestyle='--')

    plt.text(frame1_in + 10, max(df['Ball_Y']) * 0.9, f'Inbound v = {inbound_velocity:.3f} m/s', color='green')

    plt.text(frame1_out + 10, max(df['Ball_Y']) * 0.85, f'Outbound v = {outbound_velocity:.3f} m/s', color='red')

    plt.text(contact_start + (contact_end - contact_start) / 2, max(df['Ball_Y']) * 0.95, f'COR = {cor:.3f}',

             horizontalalignment='center')

    plt.xlabel('Frame Number')

    plt.ylabel('Ball_Y Position (pixels)')

    plt.title('Ball Motion: Inbound, Outbound, and COR')

    plt.legend()

    plt.grid(True)

    plt.gca().invert_yaxis()

    plt.show()

# Run

plot_ball_motion("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop1.csv", FPS, MM_PER_PIXEL)


DROP 2

In [ ]:
import pandas as pd

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def calculate_metrics(file_path, fps, mm_per_pixel):

    # Load data from CSV file

    try:

        df = pd.read_csv(file_path)

    except FileNotFoundError:

        print(f"Error: File '{file_path}' not found. Please ensure the CSV file exists.")

        return None

    except Exception as e:

        print(f"Error loading file: {str(e)}")

        return None

    # Identify contact frames based on Contact_Length

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    if contact_frames.empty:

        print("No contact frames detected.")

        return None

    contact_start = contact_frames.min()  # Frame 1712

    contact_end = contact_frames.max()    # Frame 1805

    print(f"Contact detected from frame {contact_start} to {contact_end}")

    # Inbound Velocity (before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    if len(pre_contact_df) < 100:

        print("Not enough pre-contact frames for a 100-frame window.")

        return None

    # Use a 100-frame window before contact

    frame1 = pre_contact_df['Frame'].iloc[-100]  # 100 frames before contact

    frame2 = pre_contact_df['Frame'].iloc[-1]    # Last frame before contact

    y1 = pre_contact_df[pre_contact_df['Frame'] == frame1]['Ball_Y'].iloc[0]

    y2 = pre_contact_df[pre_contact_df['Frame'] == frame2]['Ball_Y'].iloc[0]

    t1 = frame1 / fps

    t2 = frame2 / fps

    delta_t = t2 - t1

    delta_y = (y2 - y1) * mm_per_pixel / 1000  # Convert to meters (positive downward)

    inbound_velocity = delta_y / delta_t  # m/s, positive for downward

    if inbound_velocity <= 0:

        print("Warning: Inbound velocity is not positive, possibly insufficient motion detected.")

        frame1 = pre_contact_df['Frame'].iloc[0]

        delta_t = (frame2 - frame1) / fps

        y1 = pre_contact_df[pre_contact_df['Frame'] == frame1]['Ball_Y'].iloc[0]

        delta_y = (y2 - y1) * mm_per_pixel / 1000

        inbound_velocity = delta_y / delta_t

        print(f"Fallback Inbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={inbound_velocity:.3f} m/s")

    else:

        print(f"Inbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={inbound_velocity:.3f} m/s")

    # Outbound Velocity (after contact)

    post_contact_df = df[df['Frame'] > contact_end]

    available_frames = len(post_contact_df)

    if available_frames < 100:

        print(f"Not enough post-contact frames for a 100-frame window. Using {available_frames} frames instead.")

        if available_frames < 10:

            print("Too few post-contact frames (< 10) to calculate outbound velocity.")

            return None

        # Use all available frames or a 10-frame window, whichever is smaller

        window_size = min(available_frames, 10)

        frame1 = post_contact_df['Frame'].iloc[0]    # First frame after contact

        frame2 = post_contact_df['Frame'].iloc[window_size - 1]  # Last frame in window

    else:

        frame1 = post_contact_df['Frame'].iloc[0]    # First frame after contact

        frame2 = post_contact_df['Frame'].iloc[99]   # 100 frames after

    y1 = post_contact_df[post_contact_df['Frame'] == frame1]['Ball_Y'].iloc[0]

    y2 = post_contact_df[post_contact_df['Frame'] == frame2]['Ball_Y'].iloc[0]

    t1 = frame1 / fps

    t2 = frame2 / fps

    delta_t = t2 - t1

    delta_y = (y2 - y1) * mm_per_pixel / 1000  # Convert to meters (negative upward)

    outbound_velocity = delta_y / delta_t  # m/s, negative for upward

    if delta_y >= 0:

        print("Warning: Outbound velocity should be negative (upward motion, y decreasing).")

        frame2 = post_contact_df['Frame'].iloc[-1]

        y2 = post_contact_df[post_contact_df['Frame'] == frame2]['Ball_Y'].iloc[0]

        delta_t = (frame2 - frame1) / fps

        delta_y = (y2 - y1) * mm_per_pixel / 1000

        outbound_velocity = delta_y / delta_t

        print(f"Fallback Outbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={outbound_velocity:.3f} m/s")

    else:

        print(f"Outbound: Frame {frame1} (y={y1}) to Frame {frame2} (y={y2}), Δy={delta_y:.4f} m, Δt={delta_t:.4f} s, v={outbound_velocity:.3f} m/s")

    # Coefficient of Restitution (COR)

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    if cor > 1:

        print(f"Warning: COR ({cor:.3f}) > 1 is physically implausible. Adjusting to maximum realistic value of 1.")

        cor = 1.0

    elif cor < 0:

        print(f"Warning: COR ({cor:.3f}) < 0 is invalid. Setting to 0.")

        cor = 0.0

    print(f"COR = {cor:.3f}")

    return {

        "inbound_velocity": inbound_velocity,

        "outbound_velocity": outbound_velocity,

        "cor": cor

    }

# Specify the file path for Drop2

file_path = "/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop2.csv"

# Run calculation

result = calculate_metrics(file_path, FPS, MM_PER_PIXEL)

# Print results

if result:

    print("\nResults:")

    print(f"Inbound Velocity: {result['inbound_velocity']:.3f} m/s (downward)")

    print(f"Outbound Velocity: {result['outbound_velocity']:.3f} m/s (upward)")

    print(f"Coefficient of Restitution (COR): {result['cor']:.3f}")


DROP 2 GRAPH

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def plot_ball_motion(file_path, fps, mm_per_pixel):

    # Load data from CSV file

    df = pd.read_csv(file_path)

    # Identify contact frames

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    contact_start = contact_frames.min()

    contact_end = contact_frames.max()

    # Inbound Velocity (100 frames before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    if len(pre_contact_df) < 100:

        print("Not enough pre-contact frames for a 100-frame window.")

        return

    frame1_in = pre_contact_df['Frame'].iloc[-100]

    frame2_in = pre_contact_df['Frame'].iloc[-1]

    y1_in = pre_contact_df[pre_contact_df['Frame'] == frame1_in]['Ball_Y'].iloc[0]

    y2_in = pre_contact_df[pre_contact_df['Frame'] == frame2_in]['Ball_Y'].iloc[0]

    delta_t_in = (frame2_in - frame1_in) / fps

    delta_y_in = (y2_in - y1_in) * mm_per_pixel / 1000

    inbound_velocity = delta_y_in / delta_t_in

    # Outbound Velocity (adjustable window after contact)

    post_contact_df = df[df['Frame'] > contact_end]

    available_frames = len(post_contact_df)

    if available_frames < 100:

        print(f"Not enough post-contact frames for a 100-frame window. Using {available_frames} frames instead.")

        if available_frames < 10:

            print("Too few post-contact frames (< 10) to calculate outbound velocity.")

            return

        # Use all available frames or a 10-frame window, whichever is smaller

        window_size = min(available_frames, 10)

        frame1_out = post_contact_df['Frame'].iloc[0]

        frame2_out = post_contact_df['Frame'].iloc[window_size - 1]

    else:

        frame1_out = post_contact_df['Frame'].iloc[0]

        frame2_out = post_contact_df['Frame'].iloc[99]

    y1_out = post_contact_df[post_contact_df['Frame'] == frame1_out]['Ball_Y'].iloc[0]

    y2_out = post_contact_df[post_contact_df['Frame'] == frame2_out]['Ball_Y'].iloc[0]

    delta_t_out = (frame2_out - frame1_out) / fps

    delta_y_out = (y2_out - y1_out) * mm_per_pixel / 1000

    outbound_velocity = delta_y_out / delta_t_out

    # COR

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    # Plot

    plt.figure(figsize=(12, 6))

    plt.plot(df['Frame'], df['Ball_Y'], 'b-', label='Ball Position (Ball_Y)')

    plt.axvspan(contact_start, contact_end, color='gray', alpha=0.3, label='Contact Period')

    plt.axvline(frame1_in, color='green', linestyle='--', label='Inbound Start')

    plt.axvline(frame2_in, color='green', linestyle='--')

    plt.axvline(frame1_out, color='red', linestyle='--', label='Outbound Start')

    plt.axvline(frame2_out, color='red', linestyle='--')

    plt.text(frame1_in + 10, max(df['Ball_Y']) * 0.9, f'Inbound v = {inbound_velocity:.3f} m/s', color='green')

    plt.text(frame1_out + 10, max(df['Ball_Y']) * 0.85, f'Outbound v = {outbound_velocity:.3f} m/s', color='red')

    plt.text(contact_start + (contact_end - contact_start) / 2, max(df['Ball_Y']) * 0.95, f'COR = {cor:.3f}',

             horizontalalignment='center')

    plt.xlabel('Frame Number')

    plt.ylabel('Ball_Y Position (pixels)')

    plt.title('Ball Motion: Inbound, Outbound, and COR')

    plt.legend()

    plt.grid(True)

    plt.gca().invert_yaxis()

    plt.show()

# Run with Drop2 file path

plot_ball_motion("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop2.csv", FPS, MM_PER_PIXEL)


GRAPH 3

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def calculate_and_plot_cor(file_path, fps, mm_per_pixel):

    # Load data from CSV file

    df = pd.read_csv(file_path)

    # Identify contact frames

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    contact_start = contact_frames.min()  # 1655

    contact_end = contact_frames.max()    # 1728

    print(f"Contact detected from frame {contact_start} to {contact_end}")

    # Inbound Velocity (100 frames before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    if len(pre_contact_df) < 100:

        print("Not enough pre-contact frames for a 100-frame window.")

        return None

    frame1_in = pre_contact_df['Frame'].iloc[-100]  # 1555

    frame2_in = pre_contact_df['Frame'].iloc[-1]    # 1654

    y1_in = pre_contact_df[pre_contact_df['Frame'] == frame1_in]['Ball_Y'].iloc[0]

    y2_in = pre_contact_df[pre_contact_df['Frame'] == frame2_in]['Ball_Y'].iloc[0]

    delta_t_in = (frame2_in - frame1_in) / fps

    delta_y_in = (y2_in - y1_in) * mm_per_pixel / 1000

    inbound_velocity = delta_y_in / delta_t_in

    print(f"Inbound: Frame {frame1_in} (y={y1_in}) to Frame {frame2_in} (y={y2_in}), Δy={delta_y_in:.4f} m, Δt={delta_t_in:.4f} s, v={inbound_velocity:.3f} m/s")

    # Outbound Velocity (use all available post-contact frames)

    post_contact_df = df[df['Frame'] > contact_end]

    available_frames = len(post_contact_df)

    if available_frames < 10:

        print("Too few post-contact frames (< 10) to calculate outbound velocity.")

        return None

    print(f"Not enough post-contact frames for a 100-frame window. Using {available_frames} frames instead.")

    frame1_out = post_contact_df['Frame'].iloc[0]   # 1729

    frame2_out = post_contact_df['Frame'].iloc[-1]  # 1823

    y1_out = post_contact_df[post_contact_df['Frame'] == frame1_out]['Ball_Y'].iloc[0]

    y2_out = post_contact_df[post_contact_df['Frame'] == frame2_out]['Ball_Y'].iloc[0]

    delta_t_out = (frame2_out - frame1_out) / fps

    delta_y_out = (y2_out - y1_out) * mm_per_pixel / 1000

    outbound_velocity = delta_y_out / delta_t_out

    print(f"Outbound: Frame {frame1_out} (y={y1_out}) to Frame {frame2_out} (y={y2_out}), Δy={delta_y_out:.4f} m, Δt={delta_t_out:.4f} s, v={outbound_velocity:.3f} m/s")

    # COR

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    if cor > 1:

        print(f"Warning: COR ({cor:.3f}) > 1 is physically implausible. Adjusting to maximum realistic value of 1.")

        cor = 1.0

    elif cor < 0:

        print(f"Warning: COR ({cor:.3f}) < 0 is invalid. Setting to 0.")

        cor = 0.0

    print(f"COR = {cor:.3f}")


# Run with Drop3 file path

result = calculate_and_plot_cor("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop3.csv", FPS, MM_PER_PIXEL)

# Print results

if result:

    print("\nResults:")

    print(f"Inbound Velocity: {result['inbound_velocity']:.3f} m/s (downward)")

    print(f"Outbound Velocity: {result['outbound_velocity']:.3f} m/s (upward)")

    print(f"Coefficient of Restitution (COR): {result['cor']:.3f}")


DROP 3 GRAPH

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def calculate_and_plot_line(file_path, fps, mm_per_pixel):

    # Load data from CSV file

    df = pd.read_csv(file_path)

    # Identify contact frames

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    contact_start = contact_frames.min()  # 1655

    contact_end = contact_frames.max()    # 1728

    print(f"Contact detected from frame {contact_start} to {contact_end}")

    # Inbound Velocity (100 frames before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    if len(pre_contact_df) < 100:

        print("Not enough pre-contact frames for a 100-frame window.")

        return None

    frame1_in = pre_contact_df['Frame'].iloc[-100]  # 1555

    frame2_in = pre_contact_df['Frame'].iloc[-1]    # 1654

    y1_in = pre_contact_df[pre_contact_df['Frame'] == frame1_in]['Ball_Y'].iloc[0]

    y2_in = pre_contact_df[pre_contact_df['Frame'] == frame2_in]['Ball_Y'].iloc[0]

    delta_t_in = (frame2_in - frame1_in) / fps

    delta_y_in = (y2_in - y1_in) * mm_per_pixel / 1000

    inbound_velocity = delta_y_in / delta_t_in

    print(f"Inbound: Frame {frame1_in} (y={y1_in}) to Frame {frame2_in} (y={y2_in}), Δy={delta_y_in:.4f} m, Δt={delta_t_in:.4f} s, v={inbound_velocity:.3f} m/s")

    # Outbound Velocity (use all available post-contact frames)

    post_contact_df = df[df['Frame'] > contact_end]

    available_frames = len(post_contact_df)

    if available_frames < 10:

        print("Too few post-contact frames (< 10) to calculate outbound velocity.")

        return None

    print(f"Not enough post-contact frames for a 100-frame window. Using {available_frames} frames instead.")

    frame1_out = post_contact_df['Frame'].iloc[0]   # 1729

    frame2_out = post_contact_df['Frame'].iloc[-1]  # 1823

    y1_out = post_contact_df[post_contact_df['Frame'] == frame1_out]['Ball_Y'].iloc[0]

    y2_out = post_contact_df[post_contact_df['Frame'] == frame2_out]['Ball_Y'].iloc[0]

    delta_t_out = (frame2_out - frame1_out) / fps

    delta_y_out = (y2_out - y1_out) * mm_per_pixel / 1000

    outbound_velocity = delta_y_out / delta_t_out

    print(f"Outbound: Frame {frame1_out} (y={y1_out}) to Frame {frame2_out} (y={y2_out}), Δy={delta_y_out:.4f} m, Δt={delta_t_out:.4f} s, v={outbound_velocity:.3f} m/s")

    # COR

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    if cor > 1:

        print(f"Warning: COR ({cor:.3f}) > 1 is physically implausible. Adjusting to maximum realistic value of 1.")

        cor = 1.0

    elif cor < 0:

        print(f"Warning: COR ({cor:.3f}) < 0 is invalid. Setting to 0.")

        cor = 0.0

    print(f"COR = {cor:.3f}")

    # Line graph

    plt.figure(figsize=(12, 6))

    plt.plot(df['Frame'], df['Ball_Y'], 'b-', label='Ball Position (Ball_Y)')

    plt.axvspan(contact_start, contact_end, color='gray', alpha=0.3, label='Contact Period')

    plt.axvline(frame1_in, color='green', linestyle='--', label='Inbound Window')

    plt.axvline(frame2_in, color='green', linestyle='--')

    plt.axvline(frame1_out, color='red', linestyle='--', label='Outbound Window')

    plt.axvline(frame2_out, color='red', linestyle='--')

    plt.text(frame1_in + 10, max(df['Ball_Y']) * 0.9, f'Inbound v = {inbound_velocity:.3f} m/s', color='green')

    plt.text(frame1_out + 10, max(df['Ball_Y']) * 0.85, f'Outbound v = {outbound_velocity:.3f} m/s', color='red')

    plt.text(contact_start + (contact_end - contact_start) / 2, max(df['Ball_Y']) * 0.95, f'COR = {cor:.3f}',

             horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    plt.xlabel('Frame Number')

    plt.ylabel('Ball_Y Position (pixels)')

    plt.title('Ball Motion: Inbound, Outbound, and Coefficient of Restitution (Drop3)')

    plt.legend()

    plt.grid(True)

    plt.gca().invert_yaxis()  # Invert y-axis since higher Ball_Y means downward motion

    plt.show()

    return {

        "inbound_velocity": inbound_velocity,

        "outbound_velocity": outbound_velocity,

        "cor": cor

    }

# Run with Drop3 file path

result = calculate_and_plot_line("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop3.csv", FPS, MM_PER_PIXEL)

# Print results

if result:

    print("\nResults:")

    print(f"Inbound Velocity: {result['inbound_velocity']:.3f} m/s (downward)")

    print(f"Outbound Velocity: {result['outbound_velocity']:.3f} m/s (upward)")

    print(f"Coefficient of Restitution (COR): {result['cor']:.3f}")


Comparism

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

# Constants

FPS = 10000  # Frames per second

MM_PER_PIXEL = 0.5  # mm per pixel

def calculate_metrics(df, drop_name):

    # Identify contact frames

    contact_frames = df[df['Contact_Length'] > 0]['Frame']

    contact_start = contact_frames.min()

    contact_end = contact_frames.max()

    # Inbound Velocity (100 frames before contact)

    pre_contact_df = df[df['Frame'] < contact_start]

    if len(pre_contact_df) < 100:

        print(f"Not enough pre-contact frames for {drop_name}.")

        return None, None, None, None, None

    frame1_in = pre_contact_df['Frame'].iloc[-100]

    frame2_in = pre_contact_df['Frame'].iloc[-1]

    y1_in = pre_contact_df[pre_contact_df['Frame'] == frame1_in]['Ball_Y'].iloc[0]

    y2_in = pre_contact_df[pre_contact_df['Frame'] == frame2_in]['Ball_Y'].iloc[0]

    delta_t_in = (frame2_in - frame1_in) / FPS

    delta_y_in = (y2_in - y1_in) * MM_PER_PIXEL / 1000

    inbound_velocity = delta_y_in / delta_t_in

    # Outbound Velocity (all available post-contact frames)

    post_contact_df = df[df['Frame'] > contact_end]

    available_frames = len(post_contact_df)

    if available_frames < 10:

        print(f"Too few post-contact frames for {drop_name}.")

        return None, None, None, None, None

    frame1_out = post_contact_df['Frame'].iloc[0]

    frame2_out = post_contact_df['Frame'].iloc[-1]

    y1_out = post_contact_df[post_contact_df['Frame'] == frame1_out]['Ball_Y'].iloc[0]

    y2_out = post_contact_df[post_contact_df['Frame'] == frame2_out]['Ball_Y'].iloc[0]

    delta_t_out = (frame2_out - frame1_out) / FPS

    delta_y_out = (y2_out - y1_out) * MM_PER_PIXEL / 1000

    outbound_velocity = delta_y_out / delta_t_out

    # COR

    cor = -outbound_velocity / inbound_velocity if inbound_velocity > 0 else 0

    if cor > 1:

        cor = 1.0

    elif cor < 0:

        cor = 0.0

    return contact_start, contact_end, inbound_velocity, outbound_velocity, cor

# Load data

drop1_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop1.csv")

drop2_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop2.csv")

drop3_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Football_CSV/Drops/Drop3.csv")

# Calculate metrics

d1_start, d1_end, d1_v_in, d1_v_out, d1_cor = calculate_metrics(drop1_df, "Drop1")

d2_start, d2_end, d2_v_in, d2_v_out, d2_cor = calculate_metrics(drop2_df, "Drop2")

d3_start, d3_end, d3_v_in, d3_v_out, d3_cor = calculate_metrics(drop3_df, "Drop3")

# Line graph

plt.figure(figsize=(14, 8))

plt.plot(drop1_df['Frame'], drop1_df['Ball_Y'], 'g-', label='Drop1')

plt.plot(drop2_df['Frame'], drop2_df['Ball_Y'], 'b-', label='Drop2')

plt.plot(drop3_df['Frame'], drop3_df['Ball_Y'], 'r-', label='Drop3')

plt.axvspan(d1_start, d1_end, color='green', alpha=0.2, label='Drop1 Contact')

plt.axvspan(d2_start, d2_end, color='blue', alpha=0.2, label='Drop2 Contact')

plt.axvspan(d3_start, d3_end, color='red', alpha=0.2, label='Drop3 Contact')

# Annotate COR

max_y = max(max(drop1_df['Ball_Y']), max(drop2_df['Ball_Y']), max(drop3_df['Ball_Y']))

plt.text(d1_start + (d1_end - d1_start) / 2, max_y * 0.95, f'Drop1 COR = {d1_cor:.3f}',

         horizontalalignment='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

plt.text(d2_start + (d2_end - d2_start) / 2, max_y * 0.90, f'Drop2 COR = {d2_cor:.3f}',

         horizontalalignment='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

plt.text(d3_start + (d3_end - d3_start) / 2, max_y * 0.85, f'Drop3 COR = {d3_cor:.3f}',

         horizontalalignment='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

plt.xlabel('Frame Number')

plt.ylabel('Ball_Y Position (pixels)')

plt.title('Ball Motion Comparison: Drop1, Drop2, and Drop3')

plt.legend()

plt.grid(True)

plt.gca().invert_yaxis()  # Invert y-axis (higher Ball_Y is downward)

plt.savefig('drop_comparison.png')

# Print results

print("Drop1:")

print(f"Contact: {d1_start} to {d1_end}")

print(f"Inbound Velocity: {d1_v_in:.3f} m/s")

print(f"Outbound Velocity: {d1_v_out:.3f} m/s")

print(f"COR: {d1_cor:.3f}\n")

print("Drop2:")

print(f"Contact: {d2_start} to {d2_end}")

print(f"Inbound Velocity: {d2_v_in:.3f} m/s")

print(f"Outbound Velocity: {d2_v_out:.3f} m/s")

print(f"COR: {d2_cor:.3f}\n")

print("Drop3:")

print(f"Contact: {d3_start} to {d3_end}")

print(f"Inbound Velocity: {d3_v_in:.3f} m/s")

print(f"Outbound Velocity: {d3_v_out:.3f} m/s")

print(f"COR: {d3_cor:.3f}")
